In [1]:
import sys
import numpy as np
import csv as csv
import h5py
sys.path.append('../')
from Public_modules.Utilities import *
from Public_modules.Load_protein_mrna import *
import pandas
import matplotlib.pyplot as plt
import matplotlib as mpl
 

In [2]:
data=load_protein_mrna( filter_genes="",filter_lines="_lines_filtered_unique", field_data="Reporter intensity corrected_regbatch",only_protein=True)

read all parameters from parameters file
could not load peer data


In [3]:
data.keys()

dict_keys(['protein_intensity', 'peptide_intensity', 'peptide_meta', 'peptide_protein', 'protein_meta', 'line_meta', 'batch_mat'])

In [ ]:
%%bash
cd /Users/mirauta/Data/MS/hipsci/TMT/phenotypes/
awk  'NR>1 {print "hipsci""\t" $1 "\t" 1"\t" $10}' hipsci.proteomics.maxquant.uniprot.TMT_batch_14.20170517_peptide_Reporter\ intensity\ corrected_regbatch_lines_filtered_unique.txt\
> hipsci.proteomics.maxquant.uniprot.TMT_batch_14.20170517_peptide_lines_filtered_unique_list.txt

#head hipsci.proteomics.maxquant.uniprot.TMT_batch_14.20170517_peptide_lines_filtered_unique_genes_filtered_list.txt

In [ ]:
%%bash
cd /Users/mirauta/Data/MS/hipsci/TMT/phenotypes/
/Users/mirauta/Scripts/pogo/pogo_cmd  \
-in ./hipsci.proteomics.maxquant.uniprot.TMT_batch_14.20170517_peptide_lines_filtered_unique_list.txt \
-fasta /Users/mirauta/Data/Annotation/Ensembl_37.75/Homo_sapiens.GRCh37.75.pep.all.fa \
-gtf /Users/mirauta/Data/Annotation/Ensembl_37.75/gencode.v19.annotation.gtf
#Homo_sapiens.GRCh37.75.gtf

In [ ]:
import pandas
folder_data='/Users/mirauta/Data/MS/hipsci/TMT/phenotypes/'

peptide_genome_mapi=pandas.read_table(folder_data+"hipsci.proteomics.maxquant.uniprot.TMT_batch_14.20170517_peptide_lines_filtered_unique_list\
.bed",sep='\t',header=None)
peptide_genome_map=pandas.DataFrame(data=np.array(peptide_genome_mapi)[:,[0,3,2,5,1,1]], \
                                    index=peptide_genome_mapi.index, \
                                    columns={"chromosome", "feature_id","start", "end","overlap","strand"})#.set_index('feature_id',drop=False)
peptide_genome_map['chromosome']=np.array([c.replace('chr','')for c in peptide_genome_map['chromosome']])
peptide_genome_map['overlap']=np.zeros(len(peptide_genome_map['overlap']),dtype='object')
peptide_genome_map

bring common variants


In [ ]:
snps=pandas.concat([pandas.read_table('/Users/mirauta/Data/Genotypes/hipsci/'+chrom+'.vars',sep='\t',header=0)for chrom in np.arange(1,23).astype('U')])
snps=snps.set_index('CHR',drop=False).transpose()
snps

For each peptide see if it overalps known snps

In [ ]:
pep={}
for chrom in np.arange(1,6):
    print (chrom)
    pep[chrom]=peptide_genome_map.set_index('chromosome',drop=False).transpose()[str(chrom)].transpose().set_index('feature_id')
    snp=np.array(snps[chrom].transpose()['POS'])
    pep[chrom]['overlap']=[\
                           (snp[(s<snp)&(pep[chrom]['end'][inds]>snp)])\
                           for inds,s in enumerate(pep[chrom]['start'])]

In [ ]:
peptide_overlap=pandas.concat([pep[chrom] for chrom in pep.keys()])
peptide_overlap['overlap2']=np.array([len(o) for o in peptide_overlap['overlap']])

In [ ]:
peptide_overlap

In [ ]:
peptide_overlap.index[peptide_overlap['overlap2']==12]

In [ ]:
for n in range(101,120):
    pepseq=peptide_overlap.index[peptide_overlap['overlap2']==2][n];

    plt.plot(np.log(data['peptide_intensity'][pepseq].values),color='orange');
    pepseq=peptide_overlap.index[peptide_overlap['overlap2']==0][n];
    peptide_overlap.transpose()[pepseq]
    plt.plot(np.log(data['peptide_intensity'][pepseq].values),color='blue');
plt.show()

In [ ]:
xx1=data['peptide_intensity'][peptide_overlap.index[(peptide_overlap['overlap2']==2)&(peptide_overlap['strand']=='-')]]
xx2=data['peptide_intensity'][peptide_overlap.index[(peptide_overlap['overlap2']==0)&(peptide_overlap['strand']=='-')]]
print (xx1.shape)
print (xx2.shape)
np.nansum(xx1>0,0)[np.isfinite(np.nansum(xx1>0,0))]

In [ ]:
plt.hist([np.log(1+ np.nansum(xx1,0)[np.isfinite(np.nansum(xx1,0))]),np.log(1+np.nansum(xx2,0)[np.isfinite(np.nansum(xx2,0))])],normed=1,bins=20)
plt.show()
plt.hist([np.log(1+ np.nansum(xx1>0,0)[np.isfinite(np.nansum(xx1>0,0))]),np.log(1+np.nansum(xx2>0,0)[np.isfinite(np.nansum(xx2>0,0))])],normed=1,bins=20)
plt.show()

In [ ]:

AAAAPGASPSPGGDAAWSEAGPGPRPLAR
AIVEFLSNLAR
AYLQDLVEGMDFQGPGES
DESPEGSAADDLYSMLEEER
EQLLGWTAEALPAK
ETLQLVDSTTSYGLTGAVFSQDK
FSLPGATCLQGEGQGHLSQK
GIDYETVPINLIK
GSDPATLYGSSGVPEPHPGEPVQQR
LGADMEDVCGR
LQPEEADVPAYPK
LVIGQNGILSTPAVSCIIR
MYGAQEDGSVGEGDLSCILK
NYISAWELQR
SISVIDSPGILSGEK
SSLVTPSISK
SVSDTYDPYLDVALEIR
YLTLDGFDAMFR


In [ ]:
peptide_overlap.transpose()['AAAAPGASPSPGGDAAWSEAGPGPRPLAR']

In [ ]:
peptide_overlap